# 🐍 Practice n°1: data engineering

The objective of this session is to learn about the basics of data engineering. You
will have to explore the **Ratebeer** dataset using sql and python.


This dataset consists of beer reviews from ratebeer. The data span a period of more
than 10 years, including all ~3 million reviews up to November 2011. Each review
includes ratings in terms of five "aspects": appearance, aroma, palate, taste, and
overall impression. Reviews include product and user information, followed by each of
these five ratings, and a plaintext review. We also have reviews from beeradvocate.

*source* [*ratebeer dataset description*](https://snap.stanford.edu/data/web-RateBeer.html)

To avoid high compute time, we are going to work with a sample during the session.
Also, the data is already cleaned. 


Here are the main steps of the notebook :

1. Preparation
1. Data engineering in sql with *duckdb*
1. Data engineering in python with *pandas*

![](https://mermaid.ink/img/pako:eNqNkD1PwzAQhv-KdVMrJYjEjUKNxAAdmWCj7nC1L62F46SOI2ir_nfsoqIOIOHB96G793ntI6hOEwhobPehtugDe36RjsWjLKFbSnhKkTTzGGhN5KV0kx79bqQwlbDKc2acCQatORDTGHCNA7E8f9Cjel9O0s0Wj9PVt-q5jDt9QhXivMB63ykaBuM2UXxMkQ07yyLCGxqSFiPnjdqSjoYu6W-OWGMsJVv3V4_44ZV_83p0GhPqilX-GwYZtORbNDp-5TGhJYQttSRBxFRTg6MNEqQ7xVEcQ_e6dwpE8CNlMPbRFS0Mbjy2IBq0Q-xGR29d116GYgniCJ8g5jdlzYuac16WxYxXVQZ7ELysbuZ3sVnPbks-43V1yuBwFihOXw_WoMg)

In [ ]:
# Note for developers: to edit the mermaid diagram, use the mermaid live editor. Modify
# the url to access the live editor:
# https://mermaid.ink/img/pako:xxxxxxxxxx --> https://mermaid.live/edit#pako:xxxxxxxxxx

Similar data engineering steps will be performed in SQL and Python to make you
appreciate the difference between these 2 languages. The output of the 2 parts will
be the same, an enriched dataset that will be used in the next sessions.

## Preparation


### Install & import modules

In [ ]:
!pip install duckdb
!pip install pyarrow

In [ ]:
import pandas as pd
import duckdb

pd.set_option("display.max_columns", 100)

### Database configuration

In [ ]:
def sql(query):
    return con.execute(query).df()


con = duckdb.connect()
con.execute("PRAGMA threads=2")
con.execute("PRAGMA enable_object_cache")

### Read remote dataset

The data is in this git repository: [ML-boot-camp/ratebeer.git](https://github.com/ML-boot-camp/ratebeer.git).

The data is located in the `ratebeer/data/` folder.


In [ ]:
file_url = "https://github.com/ML-boot-camp/ratebeer/raw/master/data/ratebeer_sample_clean.parquet"

## Data engineering in SQL with `duckdb`

### Get some doc
Open the [w3schools SQL documentation](https://www.w3schools.com/sql/default.asp).

### Read data

Load the file `ratebeer_sample_clean.parquet` to extract a pandas DataFrame and
assign it the variable `table_ratebeer`.  
Hint:
 - [`pd.read_parquet`](https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html)


In [ ]:
table_ratebeer = pd.read_parquet(file_url)

#### Explore data

 Display a few reviews.

 Hint:
 - `SELECT`
 - `FROM`
 - `LIMIT`

In [ ]:
query = """
SELECT *
FROM table_ratebeer
LIMIT 5
"""
sql(query)

Display only some columns


 Hint:
 - `SELECT` column_name

In [ ]:
query = """
SELECT beer, text, rating --LINE TO BE REMOVED FOR STUDENTS
FROM table_ratebeer
LIMIT 5
"""
sql(query)

Count the total number of reviews

 Hint:
 - `COUNT`

In [ ]:
query = """
SELECT COUNT(*) --LINE TO BE REMOVED FOR STUDENTS
FROM table_ratebeer
"""
sql(query)

Count the distinct number of beer names and renames it as "Number of beer names"

 Hint:
 - `SELECT`...`AS`
 - `COUNT`
 - `DISTINCT`


In [ ]:
query = """
SELECT COUNT(DISTINCT beer) AS "Number of beer names" --LINE TO BE REMOVED FOR STUDENTS
FROM table_ratebeer
"""
sql(query)

 Display the number of reviews per beer.

 Hint:
 - `GROUP BY`
 - `COUNT`

In [ ]:
query = """
SELECT beer, COUNT(beer)
FROM table_ratebeer
GROUP BY beer
"""
sql(query)

 Display the 10 beers with the most reviews.

 Hint:
 - `GROUP BY`
 - `ORDER BY`...`DESC`

In [ ]:
query = """
SELECT beer, count(beer)
FROM table_ratebeer
GROUP BY beer
ORDER BY count(beer) DESC --LINE TO BE REMOVED FOR STUDENTS
LIMIT 10
"""
sql(query)

 Select the strongest API beers.

 Hint:
- `WHERE` 
- `LIKE`
- `ROUND`
- `AVG`

In [ ]:
query = """
SELECT style, ROUND(AVG(alcohol), 2) as avg_alcohol
FROM table_ratebeer
WHERE style LIKE '%IPA%' --LINE TO BE REMOVED FOR STUDENTS
GROUP BY style
ORDER BY avg_alcohol DESC
LIMIT 5
"""
sql(query)

#### Create reviewers table

Create a `table_reviewers` view which contains for each profile name, his number of
reviews and his average rating. 

Hint:
 - `CREATE VIEW ... AS`

In [ ]:
query = """
CREATE VIEW table_reviewers
AS 
    SELECT 
        user AS profile_name,
        COUNT(user) AS number_of_reviews, --LINE TO BE REMOVED FOR STUDENTS
        ROUND(AVG(rating), 1) AS average_rating

    FROM table_ratebeer
    GROUP BY user
"""
sql(query)

 Verify that the view contains what you want.

In [ ]:
query = """
SELECT *
FROM table_reviewers
"""
sql(query)

#### Combine tables

Join the `table_reviewers` with the `table_ratebeer`.

 Hint:
 - `JOIN`
 - `INNER`
 - `ON`

In [ ]:
query = """
SELECT 
  *
FROM table_ratebeer
INNER JOIN table_reviewers
    ON table_ratebeer.user == table_reviewers.profile_name --LINE TO BE REMOVED FOR STUDENTS
LIMIT 5
"""
sql(query)

Save that final result to a parquet file named `ratebeer_sample_enriched.parquet`.  
First, create a view of the table, name `table_ratebeer_enriched`.

 Hint:
 - `COPY`
 - `TO`
 - `FORMAT`

In [ ]:
query = """
CREATE VIEW table_ratebeer_enriched
AS 
    SELECT *
    FROM table_ratebeer
    INNER JOIN table_reviewers
        ON table_ratebeer.user == table_reviewers.profile_name
"""
sql(query)

In [ ]:
query = """
SELECT *
FROM table_ratebeer_enriched
LIMIT 5
"""
sql(query)

In [ ]:
# save data (optional)

# query = """
# COPY (SELECT * FROM table_ratebeer_enriched)
# TO '/content/ratebeer/data/df_master.parquet' (FORMAT 'parquet')
# """
# sql(query)

 GOOD JOB 👍

 ![](https://c.tenor.com/Cn6yJ4YTMJgAAAAC/good-job-clapping.gif)

## Data engineering in python with `pandas`

### Get some doc
- [pandas doc: main page](https://pandas.pydata.org/docs/index.html)
- [pandas doc: API reference](https://pandas.pydata.org/docs/reference/index.html)

### Read data

Load the file `ratebeer_sample_clean.parquet` to extract a pandas DataFrame and
assign
it the variable `df_ratebeer`.  
Hint:
- [`pd.read_parquet`](https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html)

In [ ]:
df_ratebeer = pd.read_parquet(file_url)

### Explore data

Display a few reviews.  


In [ ]:
df_ratebeer

Display the first 10 rows for some columns only : *beer*, *text* and
*rating*

Hint:
- [`pandas.DataFrame.head`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)

In [ ]:
(
    df_ratebeer[["beer", "text", "rating"]]
    .head(10)  # LINE TO BE REMOVED FOR STUDENTS
)

Display the dimensionality of the dataset.

 Hint:
 - [`pandas.DataFrame.shape`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html)

In [ ]:
df_ratebeer.shape  # LINE TO BE REMOVED FOR STUDENTS

Check if there are missing values in the data

Hint:
- [`pd.DataFrame.isnull()`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html)

In [ ]:
df_ratebeer.isnull().sum()

Generate descriptive statistics on the numerical variables.

Hint:
- [`pd.DataFrame.describe`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)

In [ ]:
(
    df_ratebeer.describe()  # LINE TO BE REMOVED FOR STUDENTS
)

Display the distinct beer names and then count the distinct number of beer names.


Hint:
- [`pd.Series.unique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html?highlight=unique#pandas.Series.unique)
- [`pd.Series.nunique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nunique.html?highlight=nunique#pandas.Series.nunique)


In [ ]:
(
    df_ratebeer.beer
    .unique()
)

In [ ]:
(
    df_ratebeer.beer
    .nunique()  # LINE TO BE REMOVED FOR STUDENTS
)

Display the number of reviews per beer.

Hint:
- [`pd.Series.value_counts`](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html)

In [ ]:
(
    df_ratebeer
    ["style"]
    .value_counts()  # LINE TO BE REMOVED FOR STUDENTS
)

Create the following dataframe :

- Keep only those columns:
  - `beer`,
  - `alcohol`,
  - `style`,
  - `user`,
  - `text`,
  - `rating_appearance`,
  - `rating_aroma`,
  - `rating_palate`,
  - `rating_taste`,
  - `rating`
- Keep only rows for which the `style` column contains the string `"Stout"`

Hint:
- [`pd.DataFrame.loc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html)
- [`pd.Series.str.contains`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html)


In [ ]:
selected_columns = [
    "beer",
    "alcohol",
    "style",
    "user",
    "text",
    "rating_appearance",
    "rating_aroma",
    "rating_palate",
    "rating_taste",
    "rating",
]

df_ratebeer_stout = (
    (df_ratebeer)
    .loc[:, selected_columns]
    .loc[lambda df: df["style"].str.contains("Stout")]  # LINE TO BE REMOVED FOR STUDENTS
    .reset_index(drop=True)
)

df_ratebeer_stout

Compute the number of occurences of each Stout beers.


In [ ]:
df_ratebeer_stout["style"].value_counts()

### Create reviewers dataframe


Create a `df_reviewers` view which contains for each profile name, his number of reviews and his average rating.

Hint:
- [`pandas.core.groupby.DataFrameGroupBy.agg`](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.core.groupby.DataFrameGroupBy.agg.html)
- [`pandas.DataFrame.round`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.round.html?highlight=round#pandas.DataFrame.round)
- [`pandas.DataFrame.reset_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html)

In [ ]:
df_reviewers = (
    df_ratebeer
    .groupby("user")
    .agg(
        number_of_reviews=('user', 'count'),
        average_rating=('rating', 'mean')  # LINE TO BE REMOVED FOR STUDENTS
    )
    .round(1)
    .reset_index()
)

df_reviewers

In [ ]:
df_ratebeer.head(2)

### Combine dataframes

Create a dataframe combining information from the **df_ratebeer** dataset and the **df_reviewers** dataset, using `merge`.

Merging is the equivalent of SQL's joining.

Hint:
- [`pd.DataFrame.merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)

In [ ]:
(
    df_ratebeer
    .merge(df_reviewers, on="user", how='inner')  # LINE TO BE REMOVED FOR STUDENTS
)

If some columns in both the left & right dataframes have the same name, you'll
obtain duplicated columns in the merge result. `pandas` adds the suffixes `_x`
and `_y` to avoid creating duplicate columns.
Use the `suffixes` argument to specify the suffixes to apply to duplicated
columns. In this example, there is no common column name in both dataframes.

We made lots of transformation to our datasets: we want to verify that all
values in the "primary keys" columns are indeed unique. Use the `validate`
argument to do so.

Generate the `df_master` dataset by merging the 2 dataframes.


In [ ]:
df_master = (
    df_ratebeer
    .merge(
        df_reviewers,
        on="user",  # LINE TO BE REMOVED FOR STUDENTS
        how='inner',
        validate="m:1"
    )
)

df_master.head(3)

Save the final result to a parquet file named `df_master.parquet`.

Hint:
- [`pd.DataFrame.to_parquet`](https://pandas.pydata.org/pandas-docs/version/1.1.5/reference/api/pandas.DataFrame.to_parquet.html)

In [ ]:
# Uncomment the line below to save the dataset to disk
# df_master.to_parquet("df_master.parquet")

GOOD JOB 👍

![](https://c.tenor.com/PgfvhIRWfrAAAAAd/jim-carrey-yes-sir.gif)